In [ ]:
!pip install segmentation-models-pytorch
!pip install -U git+https://github.com/albumentations-team/albumentations

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 KB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 KB 760.6 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 KB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 12.2 MB/s eta 0:00:00
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16444 sha256=58bb615e122c77c58c7b3d1907751919c7c5b4f5e75b81cd95272f2c31ef329d
  Stored in directory: /root/.cache/pip/wheels/29/16/24/752e89d88d333af39a288421e64d613b5f652918e39ef1f8e3
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60962 sha256=7d77bedf6d0d822b01ce668086744720d7ddd24b3163fe89b1f83cd5074b47b9
  Stored in directory: /root/.cache/pip/

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch 
import cv2

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 


from tqdm import tqdm

import os 
import copy
import glob


In [ ]:
CSV_FILE = '/content/Human-Segmentation-Dataset-master/train.csv' 
#says what picture corresponds to which mask
DATA_DIR = '/content/'
# where our data is stored

DEVICE = 'cuda' #Cuda as using GPU

EPOCHS = 50 #25 training iterations
LR = 0.001 #decay learning rate
IMAGE_SIZE = 320
HEIGHT = 288
WIDTH = 480
BATCH_SIZE = 4
NO_OF_IMAGES = 100
# Images are irregular shape so need to resize

ENCODER = 'resnet34' 
WEIGHTS = 'imagenet' #use weights from imagenet



In [ ]:
import os 
import copy
import glob
training_masks = sorted(glob.glob(f"/content/drive/MyDrive/Dissertation/VGG_UNet/Training/8_Annotations/*.png"))
print("Number of pictures {}".format(len(training_masks)))


training_images = sorted(glob.glob(f"/content/drive/MyDrive/Dissertation/VGG_UNet/Training/Images/*.png"))
print("Number of pictures {}".format(len(training_images)))


dict_images = {'masks' : training_masks, 'images' :training_images}

train_df = pd.DataFrame(dict_images)
train_df.head()



Number of pictures 3391
Number of pictures 3391


,masks,images
0,/content/drive/MyDrive/Dissertation/VGG_UNet/T...,/content/drive/MyDrive/Dissertation/VGG_UNet/T...
1,/content/drive/MyDrive/Dissertation/VGG_UNet/T...,/content/drive/MyDrive/Dissertation/VGG_UNet/T...
2,/content/drive/MyDrive/Dissertation/VGG_UNet/T...,/content/drive/MyDrive/Dissertation/VGG_UNet/T...
3,/content/drive/MyDrive/Dissertation/VGG_UNet/T...,/content/drive/MyDrive/Dissertation/VGG_UNet/T...
4,/content/drive/MyDrive/Dissertation/VGG_UNet/T...,/content/drive/MyDrive/Dissertation/VGG_UNet/T...


In [ ]:
images = glob.glob(f"/content/drive/MyDrive/Dissertation/Augmented_images/images_17/*.png")
masks = glob.glob(f"/content/drive/MyDrive/Dissertation/Augmented_images/masks_17/*.png")

dict_images = {'masks' : masks, 'images' :images}
aug_images = pd.DataFrame(dict_images)

# train_df = pd.concat([train_df, aug_images], ignore_index=True)
# print(len(train_df))

In [ ]:
aug_images.head()

,masks,images
0,/content/drive/MyDrive/Dissertation/Augmented_...,/content/drive/MyDrive/Dissertation/Augmented_...
1,/content/drive/MyDrive/Dissertation/Augmented_...,/content/drive/MyDrive/Dissertation/Augmented_...
2,/content/drive/MyDrive/Dissertation/Augmented_...,/content/drive/MyDrive/Dissertation/Augmented_...
3,/content/drive/MyDrive/Dissertation/Augmented_...,/content/drive/MyDrive/Dissertation/Augmented_...
4,/content/drive/MyDrive/Dissertation/Augmented_...,/content/drive/MyDrive/Dissertation/Augmented_...


In [ ]:
list(aug_images.masks)
augmented_images = []
for image_path in aug_images.masks:
    path, file = os.path.split(image_path)
    file = file.replace(".png", "")
    augmented_images.append(f"/cs/student/projects1/2019/nsultana/new_data/Training1/{file}.npz")

In [ ]:
print(augmented_images)

['/cs/student/projects1/2019/nsultana/new_data/Training/Video10_frame005810.npz', '/cs/student/projects1/2019/nsultana/new_data/Training/Video10_frame007010.npz', '/cs/student/projects1/2019/nsultana/new_data/Training/Video10_frame007020.npz', '/cs/student/projects1/2019/nsultana/new_data/Training/Video10_frame011330.npz', '/cs/student/projects1/2019/nsultana/new_data/Training/Video10_frame016070.npz', '/cs/student/projects1/2019/nsultana/new_data/Training/Video10_frame017160.npz', '/cs/student/projects1/2019/nsultana/new_data/Training/Video10_frame017260.npz', '/cs/student/projects1/2019/nsultana/new_data/Training/Video10_frame017380.npz', '/cs/student/projects1/2019/nsultana/new_data/Training/Video10_frame020090.npz', '/cs/student/projects1/2019/nsultana/new_data/Training/Video10_frame020110.npz', '/cs/student/projects1/2019/nsultana/new_data/Training/Video10_frame020120.npz', '/cs/student/projects1/2019/nsultana/new_data/Training/Video10_frame020130.npz', '/cs/student/projects1/2019

In [ ]:
np.savez_compressed(f"aug_images.npz", images=augmented_images)

In [ ]:
validation_masks = sorted(glob.glob(f"/content/drive/MyDrive/Dissertation/VGG_UNet/Validation/8_Annotations/*.png"))

print("Number of pictures {}".format(len(validation_masks)))

validation_images = sorted(glob.glob(f"/content/drive/MyDrive/Dissertation/VGG_UNet/Validation/Images/*.png"))

print("Number of pictures {}".format(len(validation_images)))

dict_images = {'masks' : validation_masks, 'images' :validation_images}

valid_df = pd.DataFrame(dict_images)

valid_df.head()

Number of pictures 534
Number of pictures 534


,masks,images
0,/content/drive/MyDrive/Dissertation/VGG_UNet/V...,/content/drive/MyDrive/Dissertation/VGG_UNet/V...
1,/content/drive/MyDrive/Dissertation/VGG_UNet/V...,/content/drive/MyDrive/Dissertation/VGG_UNet/V...
2,/content/drive/MyDrive/Dissertation/VGG_UNet/V...,/content/drive/MyDrive/Dissertation/VGG_UNet/V...
3,/content/drive/MyDrive/Dissertation/VGG_UNet/V...,/content/drive/MyDrive/Dissertation/VGG_UNet/V...
4,/content/drive/MyDrive/Dissertation/VGG_UNet/V...,/content/drive/MyDrive/Dissertation/VGG_UNet/V...


In [ ]:
testing_masks = sorted(glob.glob(f"/content/drive/MyDrive/Dissertation/VGG_UNet/Testing/8_Annotations/*.png"))

print("Number of pictures {}".format(len(testing_masks)))

testing_images = sorted(glob.glob(f"/content/drive/MyDrive/Dissertation/VGG_UNet/Testing/Images/*.png"))

print("Number of pictures {}".format(len(testing_images)))

dict_images = {'masks' : testing_masks, 'images' :testing_images}

test_df = pd.DataFrame(dict_images)

test_df.head()

Number of pictures 586
Number of pictures 586


,masks,images
0,/content/drive/MyDrive/Dissertation/VGG_UNet/T...,/content/drive/MyDrive/Dissertation/VGG_UNet/T...
1,/content/drive/MyDrive/Dissertation/VGG_UNet/T...,/content/drive/MyDrive/Dissertation/VGG_UNet/T...
2,/content/drive/MyDrive/Dissertation/VGG_UNet/T...,/content/drive/MyDrive/Dissertation/VGG_UNet/T...
3,/content/drive/MyDrive/Dissertation/VGG_UNet/T...,/content/drive/MyDrive/Dissertation/VGG_UNet/T...
4,/content/drive/MyDrive/Dissertation/VGG_UNet/T...,/content/drive/MyDrive/Dissertation/VGG_UNet/T...


In [ ]:
train_df = train_df.sample(frac=1).reset_index(drop=True)
valid_df = valid_df.sample(frac=1).reset_index(drop=True)
test_df = test_df.sample(frac=1).reset_index(drop=True)



In [ ]:
train_df.to_csv("training.csv")
valid_df.to_csv("validation.csv")
test_df.to_csv("testing.csv")

In [ ]:
train_df = pd.read_csv("training.csv")
valid_df= pd.read_csv("validation.csv")
test_df =pd.read_csv("testing.csv")

In [ ]:
train_df = pd.read_csv("training (1).csv")

In [ ]:
def generateMasks(predicted_flows, next, real_mask):
  
  dim = (WIDTH, HEIGHT)
  first= True
  masks = []

  for flow in predicted_flows:
    
    flow = flow.detach().to('cpu').numpy()
    flow = np.moveaxis(flow, 0, -1)
    if next:
      flow = -flow

    h = flow.shape[0]
    w = flow.shape[1]
    
    flow[:,:,0] += np.arange(w)
    flow[:,:,1] += np.arange(h)[:,np.newaxis]

    if first:
      
      mask = real_mask
      
      mask = mask[:,:,np.newaxis]
      
      if next: 
        frame = np.squeeze(mask, axis=(-1,))
        
        temp = frame
        temp[temp == 8] = 0
        frame = torch.Tensor(frame)
        temp = torch.Tensor(temp)
        real_mask = frame
        masks.append(temp) 

      
      new_frame = cv2.remap(mask, flow, None, interpolation=cv2.INTER_NEAREST, borderMode=cv2.BORDER_CONSTANT,  borderValue=100)
      first=False

    else:
      
      new_frame = cv2.remap(new_frame, flow, None, interpolation=cv2.INTER_NEAREST, borderMode=cv2.BORDER_CONSTANT, borderValue=100 )

    frame = torch.Tensor(new_frame) 
            
    frame = frame_processing(frame)
    frame = torch.Tensor(frame)
    masks.append(frame)
  return masks
  

In [ ]:
def frame_processing(mask):
  mask[mask == 100] = -1
  temp_mask = np.copy(mask)
  temp_mask[temp_mask !=7] = 50

  kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(4,4))
  res = cv2.morphologyEx(temp_mask,cv2.MORPH_OPEN,kernel, iterations=1)
  res[res == 50] = 0
  instrument_mask = ma.make_mask(res)
  filled_mask = np.copy(mask)
  filled_mask[instrument_mask] = 0
  result = (res + filled_mask)
  return result

In [ ]:
from torchvision.models.optical_flow import Raft_Small_Weights, Raft_Large_Weights
from torchvision.models.optical_flow import raft_small, raft_large

def getMaskSequence(images, real_mask):
  weights = Raft_Large_Weights.DEFAULT
  transforms = weights.transforms()

  with torch.no_grad():
    model = raft_large(weights=Raft_Large_Weights.DEFAULT, progress=False).to('cuda')
    model = model.eval()

  masks = []
  for i in range (0,2):
    if i==0:
      # img1_batch = torch.stack(list(images[0:3])[::-1])
      # img2_batch = torch.stack(list(images[1:4])[::-1])
      img1_batch = torch.stack(list(images[0:7])[::-1])
      img2_batch = torch.stack(list(images[1:8])[::-1])
      # 
    else:
      # img1_batch = torch.stack(list(images[3:6]))
      # img2_batch = torch.stack(list(images[4:7]))
      img1_batch = torch.stack(list(images[7:10]))
      img2_batch = torch.stack(list(images[8:11]))
    
    img1_batch, img2_batch = transforms(img1_batch, img2_batch)
    with torch.no_grad():
      list_of_flows = model(img1_batch.to('cuda'), img2_batch.to('cuda'))
    
    predicted_flows = list_of_flows[-1]
    if i==0:
      new_masks = generateMasks(predicted_flows, False, real_mask)[::-1]
    else:
      new_masks = generateMasks(predicted_flows, True, real_mask)

    masks += new_masks
  maskData = torch.stack(masks)

  return maskData 

#masks = getMaskSequence(images, mask_path)
  
      

In [ ]:

def replaceNewPixels(real_mask, optical_flow):

    real_mask_labels = np.unique(real_mask)
    optical_flow_labels = np.unique(optical_flow)
    main_list = np.setdiff1d(optical_flow_labels,real_mask_labels)
   
    for value in main_list:

      optical_flow[optical_flow == value] = -1
    
    optical_flow[optical_flow == 8] = 0
    
    return optical_flow

In [ ]:

for image_path in aug_images.images:
  # path, file = os.path.split(image_path)
  # if "Augmented_images" in path:
  #   print("True")
  # file = file.replace(".png", "")
  file_path, mask_path = getFilePath(image_path, "Testing")
  print(file_path)

In [ ]:
def createImageSequence(sequences):
  images = []
  dim = (960, 544)
  #images = torch.Tensor(10, HEIGHT, WIDTH)
  for image_path in sequences:
    image = cv2.imread(image_path)
    try:
      image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    except:
      print("error", image_path)
      
    
    image = cv2.resize(image, dim)
    image = image / 255.0
    image = torch.Tensor(image) 
    image = image.permute(2, 0, 1)
    images.append(image)

  #images = torch.stack(images)
  return images

def getFilePath(image_path, split):
  path, file = os.path.split(image_path)
  file = file.replace(".png", "")
  parts = file.split("_")
  video = parts[0]
  frame = parts[1]

  #/content/drive/MyDrive/Dissertation/frames/Video1/Video1_frame000090
  file_path = f"/content/drive/MyDrive/Dissertation/frames/{video}/{video}_{frame}/*.png"
  mask_path = f"/content/drive/MyDrive/Dissertation/VGG_UNet/{split}/8_Annotations/{video}_{frame}.png"

  return file_path, mask_path


In [ ]:
import albumentations as A

import numpy.ma as ma
no_of_images = test_df.shape[0]
aug = A.Compose([
A.Resize(height=544, width=960)
])


dim = (WIDTH, HEIGHT)
# mask_dim = (960, 544)
for idx in range (0, no_of_images):
    row = test_df.iloc[idx]
    print( idx)
    image_path = row.images
    mask_path = row.masks
    file_path, mask_path = getFilePath(image_path, "Testing")
    
    file_name = os.path.split(mask_path)[1].replace(".png", "")
    
    

    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    
    sequences = sorted(glob.glob(file_path))
    sequences = sequences[3:14]
    try:
      images = createImageSequence(sequences)
      
      temp_image = images[3].numpy().transpose(2, 1, 0)
      
      temp_image = cv2.resize(temp_image, mask_dim)
      
      aug_result = aug(image=temp_image,mask=mask)
      #mask = cv2.resize(mask, dim, cv2.INTER_NEAREST)
      resized_mask = aug_result['mask']
      #resized_mask[resized_mask == 0] = 8
      masks = getMaskSequence(images, resized_mask)
      masks = masks.numpy()
      
      images = np.stack(images, axis=0)


      #np.savez_compressed(f"/content/drive/MyDrive/Testing/{file_name}", images=images, masks=masks)
      
    except Exception as e:
      print(e)
    
    




